In [1]:
import numpy as np 
import pandas as pd

In [2]:
pr = pd.read_csv('diabetes.csv')
pr.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [3]:
# corelation of output with other parameters 
# we can remove which are not important parameters

pr.corr()['Outcome']

Pregnancies                 0.221898
Glucose                     0.466581
BloodPressure               0.065068
SkinThickness               0.074752
Insulin                     0.130548
BMI                         0.292695
DiabetesPedigreeFunction    0.173844
Age                         0.238356
Outcome                     1.000000
Name: Outcome, dtype: float64

In [4]:
# in x we provide input data and in y we put output data
x = pr.iloc[:,:-1].values
y = pr.iloc[:,-1].values

In [5]:
#scaling the data (0-1)
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
x = scaler.fit_transform(x)

In [6]:
x.shape

(768, 8)

In [7]:
# to train the data 
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test = train_test_split(x,y,train_size=0.2,random_state=1)

In [8]:
#to create a model
import tensorflow
from tensorflow import keras
from tensorflow.keras import  Sequential
from tensorflow.keras.layers import  Dense

In [9]:
# create node and layer in model
model = Sequential()

model.add(Dense(32,activation='relu',input_dim = 8))
model.add(Dense(1,activation='sigmoid'))

model.compile(optimizer= 'Adam',loss='binary_crossentropy',metrics=['accuracy'])



c:\Users\dt160\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [10]:
#generate a model according to parameters

model.fit(x_train,y_train,batch_size=32,epochs=10,validation_data=(x_test,y_test))

Epoch 1/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 65ms/step - accuracy: 0.6297 - loss: 0.6423 - val_accuracy: 0.6504 - val_loss: 0.6401
Epoch 2/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.6645 - loss: 0.6368 - val_accuracy: 0.6683 - val_loss: 0.6282
Epoch 3/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.6840 - loss: 0.6154 - val_accuracy: 0.6829 - val_loss: 0.6178
Epoch 4/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.6402 - loss: 0.6084 - val_accuracy: 0.6878 - val_loss: 0.6082
Epoch 5/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.6962 - loss: 0.5763 - val_accuracy: 0.6976 - val_loss: 0.5990
Epoch 6/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.7214 - loss: 0.5695 - val_accuracy: 0.7057 - val_loss: 0.5911
Epoch 7/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.7080 - loss: 0.5747 - val_accuracy: 0.7041 - val_loss: 0.5837
Epoch 8/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.7370 - loss: 0.5428 - val_accuracy: 0.7154 - val_loss: 0.5767


In [11]:
# to automate the whole process of creating node and layer by hypertuning
import kerastuner as kt

C:\Users\dt160\AppData\Local\Temp\ipykernel_13136\1422821170.py:2: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  import kerastuner as kt


In [12]:
#creating a model for using the hypertuner
def build_model(hp):
    model = Sequential()

    model.add(Dense(32,activation='relu',input_dim=8))
    model.add(Dense(1,activation='sigmoid'))

    optimizer = hp.Choice('optimizer', values=['adam', 'sgd', 'rmsprop', 'adadelta'])

    model.compile(optimizer=optimizer,loss='binary_crossentropy',metrics=['accuracy'])

    return model

In [19]:
# tuner for model
tuner = kt.RandomSearch(build_model,objective='val_accuracy',max_trials=5)

In [20]:
# it search the best tuning setting for the best result with this model
# the weights are save in new creted folder
tuner.search(x_train,y_train,epochs=5,validation_data = (x_test,y_test))

Trial 4 Complete [00h 00m 02s]
val_accuracy: 0.40162602066993713

Best val_accuracy So Far: 0.6829268336296082
Total elapsed time: 00h 00m 08s


In [21]:
# it shows which optimizer is better for this model
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'rmsprop'}

In [22]:
# this code search the best model (high accuracy)
model = tuner.get_best_models(num_models=1)[0]

c:\Users\dt160\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 6 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [23]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 32)             │           288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 321 (1.25 KB)

 Trainable params: 321 (1.25 KB)

 Non-trainable params: 0 (0.00 B)

In [24]:
model.fit(x_train,y_train,batch_size=32,epochs=100,initial_epoch=7,validation_data=(x_test,y_test))

Epoch 8/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 54ms/step - accuracy: 0.6402 - loss: 0.5773 - val_accuracy: 0.7008 - val_loss: 0.5615
Epoch 9/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.6910 - loss: 0.5452 - val_accuracy: 0.7073 - val_loss: 0.5541
Epoch 10/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.6693 - loss: 0.5525 - val_accuracy: 0.7138 - val_loss: 0.5488
Epoch 11/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.6589 - loss: 0.5522 - val_accuracy: 0.7171 - val_loss: 0.5438
Epoch 12/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.7349 - loss: 0.5266 - val_accuracy: 0.7268 - val_loss: 0.5395
Epoch 13/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.7436 - loss: 0.5203 - val_accuracy: 0.7268 - val_loss: 0.5356
Epoch 14/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.7080 - loss: 0.5381 - val_accuracy: 0.7301 - val_loss: 0.5323
Epoch 15/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.7262 - loss: 0.5269 - val_accuracy: 0.7366 - val

In [27]:
# right number of nurons for the model
#find the num of nurons right fot this model through hyperparameter

def build_model(hp):
    model = Sequential()

    units = hp.Int('units',8,128,step=8) 

    model.add(Dense(units,activation='relu',input_dim = 8))
    model.add(Dense(1,activation='sigmoid'))

    model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])

    return model



In [28]:
tuner = kt.RandomSearch(build_model,
                        objective='val_accuracy',
                        max_trials=5
                        )

In [29]:
tuner.search(x_train,y_train,epochs=5,validation_data=(x_test,y_test))

Trial 5 Complete [00h 00m 02s]
val_accuracy: 0.6943089365959167

Best val_accuracy So Far: 0.7349593639373779
Total elapsed time: 00h 00m 10s


In [30]:
# it shows the all trial improvement and important parameters
tuner.results_summary()

Results summary
Results in .\untitled_project
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 0 summary
Hyperparameters:
units: 112
Score: 0.7349593639373779

Trial 1 summary
Hyperparameters:
units: 48
Score: 0.7170731425285339

Trial 4 summary
Hyperparameters:
units: 80
Score: 0.6943089365959167

Trial 3 summary
Hyperparameters:
units: 72
Score: 0.6682927012443542

Trial 2 summary
Hyperparameters:
units: 8
Score: 0.4032520353794098


In [31]:
# to see the best hyperparamer
tuner.get_best_hyperparameters()[0].values

{'units': 112}

In [32]:
# to see the best model from the trials
model = tuner.get_best_models(num_models=1)[0]

c:\Users\dt160\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [33]:
model.fit(x_train,y_train,batch_size=32,epochs=100,initial_epoch=6)

Epoch 7/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.7636 - loss: 0.5615  
Epoch 8/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7692 - loss: 0.5489 
Epoch 9/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7775 - loss: 0.5305 
Epoch 10/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7801 - loss: 0.5200 
Epoch 11/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7701 - loss: 0.4999  
Epoch 12/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7931 - loss: 0.4686 
Epoch 13/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7844 - loss: 0.4676 
Epoch 14/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7692 - loss: 0.4872 
Epoch 15/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7875 - loss: 0.4510 
Epoch 16/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7701 - loss: 0.4835 
Epoch 17/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7923 - loss: 0.4576 
Epoch 18/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.79

In [37]:
def build_model(hp):

    model = Sequential()

    model.add(Dense(72,activation='relu',input_dim=8))

    for i in range(hp.Int('num_layers',1,10)):
        model.add(Dense(72,activation='relu'))

    model.add(Dense(1,activation='sigmoid'))    

    model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])

    return model

    

In [38]:
tuner = kt.RandomSearch(build_model,objective='val_accuracy',max_trials=3,directory='mydir',project_name='num_layer')

In [39]:
tuner.search(x_train,y_train,epochs=5,validation_data=(x_test,y_test))

Trial 3 Complete [00h 00m 04s]
val_accuracy: 0.7349593639373779

Best val_accuracy So Far: 0.7430894374847412
Total elapsed time: 00h 00m 11s


In [42]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 9}

In [43]:
model = tuner.get_best_models(num_models=1)[0]
model.fit(x_train,y_train,epochs=100,initial_epoch=6,validation_data=(x_test,y_test))

Epoch 7/100


c:\Users\dt160\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\dt160\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 46 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


5/5 ━━━━━━━━━━━━━━━━━━━━ 3s 75ms/step - accuracy: 0.7957 - loss: 0.5838 - val_accuracy: 0.7480 - val_loss: 0.5483
Epoch 8/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.7527 - loss: 0.4943 - val_accuracy: 0.7252 - val_loss: 0.5223
Epoch 9/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.7884 - loss: 0.4415 - val_accuracy: 0.7561 - val_loss: 0.5438
Epoch 10/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.8140 - loss: 0.3730 - val_accuracy: 0.7268 - val_loss: 0.5752
Epoch 11/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.7931 - loss: 0.4233 - val_accuracy: 0.7398 - val_loss: 0.5606
Epoch 12/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.8501 - loss: 0.3683 - val_accuracy: 0.7333 - val_loss: 0.5525
Epoch 13/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.8727 - loss: 0.3543 - val_accuracy: 0.7382 - val_loss: 0.5633
Epoch 14/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - accuracy: 0.8666 - loss: 0.3371 - val_accuracy: 0.7398 - val_loss: 0.5856